In [1]:
from __future__ import print_function
import itertools
import pandas as pd
import numpy as np
import os
import logging
import glob
import networkx as nx
import math
from tqdm import tqdm

/Users/rajeevkumar/anaconda/lib/python3.4/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
sampdirs = glob.glob('../../sailfish_quant/*')
auxDir = "aux"
cutoff = 0.5
netfile = 'netfile'
# output = rapObj.buildNetFile(sampdirs, netfile, cutoff, auxDir, False)
ofile ='filterGraph_svm.txt'
expDict = {
    'scramble': {'SRR493366_quant': '../../sailfish_quant/SRR493366_quant',
                 'SRR493367_quant': '../../sailfish_quant/SRR493367_quant',
                 'SRR493368_quant': '../../sailfish_quant/SRR493368_quant'},
    'HOXA1KD': {'SRR493369_quant': '../../sailfish_quant/SRR493369_quant',
                'SRR493370_quant': '../../sailfish_quant/SRR493370_quant',
                'SRR493371_quant': '../../sailfish_quant/SRR493371_quant'}
}

In [3]:
class EquivCollection(object):
    def __init__(self):
        self.tnames = []
        self.eqClasses = {}
        self.hasNames = False

    def setNames(self, names):
        self.tnames = names
        self.hasNames = True

    def add(self, tids, count):
        if tids in self.eqClasses:
            self.eqClasses[tids] += count
        else:
            self.eqClasses[tids] = count


In [4]:
def readEqClass(eqfile, eqCollection):
    with open(eqfile) as ifile:
        numTran = int(ifile.readline().rstrip())
        numEq = int(ifile.readline().rstrip())
        print("\nfile: {}; # tran = {}; # eq = {}".format(eqfile, numTran, numEq))
        if not eqCollection.hasNames:
            tnames = []
            for i in range(numTran):
                tnames.append(ifile.readline().rstrip())
            eqCollection.setNames(tnames)
        else:
            for i in range(numTran):
                ifile.readline()

        for i in range(numEq):
            toks = list(map(int, ifile.readline().rstrip().split('\t')))
            nt = toks[0]
            tids = tuple(toks[1:-1])
            count = toks[-1]
            eqCollection.add(tids, count)


In [5]:
def getCountsFromEquiv(eqCollection):
        countDict = {}
        tn = eqCollection.tnames
        for tids, count in eqCollection.eqClasses.items():
            for t in tids:
                if tn[t] in countDict:
                    countDict[tn[t]] += count
                else:
                    countDict[tn[t]] = count
        # ensure no division by 0
        for t in eqCollection.tnames:
            if t in countDict:
                countDict[t] += 1.0
            else:
                countDict[t] = 1.0
        return countDict

In [6]:
# Get just the set of condition names
conditions = expDict.keys()
logging.info("conditions = {}".format(conditions))
eqClasses = {}
for cond in conditions:
    print(expDict[cond])
    for sampNum, sampPath in expDict[cond].items():
        if cond not in eqClasses:
            eqClasses[cond] = EquivCollection()
        eqPath = os.path.sep.join([sampPath, auxDir, "eq_classes.txt"])
        readEqClass(eqPath, eqClasses[cond])

{'SRR493366_quant': '../../sailfish_quant/SRR493366_quant', 'SRR493367_quant': '../../sailfish_quant/SRR493367_quant', 'SRR493368_quant': '../../sailfish_quant/SRR493368_quant'}

file: ../../sailfish_quant/SRR493366_quant/aux/eq_classes.txt; # tran = 107389; # eq = 95472

file: ../../sailfish_quant/SRR493367_quant/aux/eq_classes.txt; # tran = 107389; # eq = 98035

file: ../../sailfish_quant/SRR493368_quant/aux/eq_classes.txt; # tran = 107389; # eq = 101801
{'SRR493371_quant': '../../sailfish_quant/SRR493371_quant', 'SRR493370_quant': '../../sailfish_quant/SRR493370_quant', 'SRR493369_quant': '../../sailfish_quant/SRR493369_quant'}

file: ../../sailfish_quant/SRR493371_quant/aux/eq_classes.txt; # tran = 107389; # eq = 104868

file: ../../sailfish_quant/SRR493370_quant/aux/eq_classes.txt; # tran = 107389; # eq = 102891

file: ../../sailfish_quant/SRR493369_quant/aux/eq_classes.txt; # tran = 107389; # eq = 100141


In [7]:
# ambigCounts = {cond : getCountsFromEquiv(eqClasses[cond]) for cond in conditions}

# sailfish = {}
# for cond in conditions:
#     sailfish[cond] = ambigCounts[cond]

In [8]:
# ambigCounts

In [9]:
# # ambigCounts = {cond : getCountsFromEquiv(eqClasses[cond]) for cond in conditions}

# # sailfish = {}
# # for cond in conditions:
# #     sailfish[cond] = ambigCounts[cond]

# logging.info("Done Reading")
# count = 0
# numTrimmed = 0
# with open(netfile) as f, open(ofile, 'w') as ofile:
#     data = pd.read_table(f, header=None)
#     print(len(data))
#     for i in tqdm(range(100)):
#         count += 1
#         #print("\r{} done".format(count), end="")
#         #Alternative hypo
#         x = data[0][i]
#         print('x : ', x)
#         y = data[1][i]
#         print('y :', y)
#         non_null=0
#         x_all=0
#         y_all=0
#         # Calculating l1
#         for cond in conditions:
#             y_g = sailfish[cond][y]
#             print('y_g', y_g)
#             x_g = sailfish[cond][x]
#             print('x_g', x_g)
#             r = y_g / x_g
#             print('r', r)
#             non_null += (y_g * math.log(r*x_g)) - (r*x_g)
# #             print('non_null',non_null)
#             non_null += (x_g * math.log(x_g)) - x_g
#             print('non_null',non_null)
#             x_all += x_g
#             print('x_all',x_all)
#             y_all += y_g
#             print('y_all', y_all)
#         #null hypothesis
#         null = 0
#         r_all = y_all / x_all
#         # Calculating l0
#         print('.......')
#         for cond in conditions:
#             y_g = sailfish[cond][y]
#             print('y_g', y_g)
#             x_g = sailfish[cond][x]
#             print('x_g', x_g)
#             mean_x = (x_g + y_g) / (1+r_all)
#             null += (y_g * math.log(r_all * mean_x)) - (r_all * mean_x)
#             null += (x_g * math.log(mean_x)) - mean_x
#             print('null : ', null)
            
#         D = 2*(non_null-null)
#         print('D', D)
#         print('label', data[2][i])
#         if D <= 20:
#             ofile.write("{}\t{}\t{}\n".format(x, y, data[2][i]))
#         else:
#             numTrimmed += 1
# logging.info("Trimmed {} edges".format(numTrimmed))


In [10]:
# data = None
# with open(netfile) as f:
#     data = pd.read_table(f, header=None)

In [11]:
# data

In [12]:
sep = os.path.sep
sffiles = [sep.join([sd, 'quant.sf']) for sd in sampdirs]
eqfiles = [sep.join([sd, auxDir, 'eq_classes.txt']) for sd in sampdirs]

In [13]:
# quant = pd.read_table(sffiles[0])
# quant.set_index('Name', inplace=True)
# quant

In [14]:
# eqfiles[0]

In [15]:
# eqfile = eqfiles[0]
# numSamp = 0
# firstSamp = True
# tnames = []
# weightDict = {}
# diagCounts = None
# sumCounts = None
# ambigCounts = None
# firstSamp = True
# numSamp = 0
# tot = 0
# eqClasses = {}

# with open(eqfile) as ifile:
#     numSamp += 1
#     numTran = int(ifile.readline().rstrip())
#     numEq = int(ifile.readline().rstrip())
# #     logging.info("quant file: {}; eq file: {}; # tran = {}; # eq = {}".format(sffile, eqfile, numTran, numEq))
#     if firstSamp:
#         for i in range(numTran):
#             tnames.append(ifile.readline().rstrip())
#         diagCounts = np.zeros(len(tnames))
#         sumCounts = np.zeros(len(tnames))
#         ambigCounts = np.zeros(len(tnames))
#     else:
#         for i in range(numTran):
#             ifile.readline()

#     # for easy access to quantities of interest
#     tpm = quant.loc[tnames, 'TPM'].values
#     estCount = quant.loc[tnames, 'NumReads'].values
#     efflens = quant.loc[tnames, 'EffectiveLength'].values
#     sumCounts = np.maximum(sumCounts, estCount)

#     for i in range(10): #numEq
#         toks = list(map(int, ifile.readline().rstrip().split('\t')))
#         print(toks)
#         nt = toks[0]
#         print('nt', nt)
#         tids = tuple(toks[1:-1])
#         print('tids', tids)
#         count = toks[-1]
#         print('count', count)
#         if tids in eqClasses:
#             eqClasses[tids] += count
#         else:
#             eqClasses[tids] = count

#         # Add the contribution to the graph
#         denom = sum([tpm[t] for t in tids])
#         for t1, t2 in itertools.combinations(tids,2):
#             tpm1 = tpm[t1]
#             tpm2 = tpm[t2]
#             w = count * ((tpm1 + tpm2) / denom)
#             key = (t1, t2)
#             if key in weightDict:
#                 weightDict[key] += w
#             else:
#                 weightDict[key] = w
#         for t in tids:
#             diagCounts[t] += count * (tpm[t] / denom)
#             ambigCounts[t] += count
#     firstSamp = False

In [16]:
# tnames = []
# with open(eqfile) as ifile:
#     for i in range(10):
#         tnames.append(ifile.readline().rstrip())
#     eqCollection.setNames(tnames)

In [17]:
netfile = pd.read_csv('netfile', sep='\t')

In [18]:
# netfile

In [19]:
# eqfile = eqfiles[0]
# numSamp = 0
# firstSamp = True
# tnames = []
# firstSamp = True
# numSamp = 0
# eqClasses = {}

# dict_pairTids_count = {} 
# dict_eqClass = {}
# dic_uniq_transcripts = {}

# with open(eqfile) as ifile:
#     numSamp += 1
#     numTran = int(ifile.readline().rstrip())
#     numEq = int(ifile.readline().rstrip())
#     if firstSamp:
#         for i in range(numTran):
#             tnames.append(ifile.readline().rstrip())
#     else:
#         for i in range(numTran):
#             ifile.readline()

#     counter = 1
#     for i in range(numEq): #numEq
#         toks = list(map(int, ifile.readline().rstrip().split('\t')))
#         tids = tuple(toks[1:-1])
# #         print(tids)
#         if tids not in dict_eqClass:
#             dict_eqClass[tids] = i+1
#             for ids in tids:
#                 if ids not in dic_uniq_transcripts:
#                     dic_uniq_transcripts[ids] = set([i+1])
#                 else:
# #                     print('duplicate')
#                     set_counter = dic_uniq_transcripts[ids]
#                     set_counter.add(i+1)
#                     dic_uniq_transcripts[ids] = set_counter
                    

In [20]:
# dict_eqClass.keys()

In [21]:
# lst_uniq_tids = dic_uniq_transcripts.keys()
# dict_prob = {}
# lst_keys = dict_eqClass.keys()
# for tup in lst_keys:
#     for t1,t2 in itertools.combinations(tup,2):
#         set_t1 = dic_uniq_transcripts[t1]
#         set_t2 = dic_uniq_transcripts[t2]
#         length_intersection = len(set_t1.intersection(set_t2))
#         length_union = len(set_t1.union(set_t2))
#         if length_intersection > 0:
#             dict_prob[(t1,t2)] = length_intersection / length_union

In [22]:
# from operator import itemgetter
# sorted(dict_prob.items(), key=itemgetter(1), reverse=False)

In [23]:
# netfile = pd.read_csv('netfile', sep='\t')

In [24]:
# len(dict_prob)

In [25]:
len(netfile)

244220

In [26]:
conditions = expDict.keys()
conditions

dict_keys(['scramble', 'HOXA1KD'])

In [27]:
# # code analysis Part 1
# conditions = expDict.keys()
# for cond in conditions:
#     print(expDict[cond])
    
#     for sampNum, sampPath in expDict[cond].items():
        

In [28]:
# condition1_paths = expDict['scramble'].values()
# condition1_paths

In [39]:
def process_data(eqClass_counter, eqfile, numSamp, firstSamp, tnames, dict_eqClass, dic_uniq_transcripts):
    print('Entered function')
    with open(eqfile) as ifile:
        numSamp += 1
        numTran = int(ifile.readline().rstrip())
        numEq = int(ifile.readline().rstrip())
        if firstSamp:
            for i in range(numTran):
                tnames.append(ifile.readline().rstrip())
        else:
            for i in range(numTran):
                ifile.readline()
        for i in range(numEq): #numEq
            eqClass_counter += 1
            toks = list(map(int, ifile.readline().rstrip().split('\t')))
            tids = tuple(toks[1:-1])
            count = toks[-1]
            if tids not in dict_eqClass:
                dict_eqClass[tids] = count    #eqClass_counter
                for ids in tids:
                    if ids not in dic_uniq_transcripts:
                        dic_uniq_transcripts[ids] = set([eqClass_counter])
                    else:
    #                     print('duplicate')
                        set_counter = dic_uniq_transcripts[ids]
                        set_counter.add(eqClass_counter)
                        dic_uniq_transcripts[ids] = set_counter


In [51]:
# Condition 1: dump

# condition1_paths = expDict['scramble'].values()
condition1_paths = expDict['HOXA1KD'].values()
dict_eqClass = {}
dic_uniq_transcripts = {}
eqClass_counter = 0
for path in condition1_paths:
    # initialize
    tnames = []
    firstSamp = True
    numSamp = 0
    eqfile = os.path.sep.join([path, auxDir, "eq_classes.txt"])
    print(eqfile)
    process_data(eqClass_counter, eqfile, numSamp, firstSamp, tnames, dict_eqClass, dic_uniq_transcripts)
#     [dict_eqClass, dic_uniq_transcripts, eqClass_counter] = 
    


../../sailfish_quant/SRR493370_quant/aux/eq_classes.txt
Entered function
../../sailfish_quant/SRR493369_quant/aux/eq_classes.txt
Entered function
../../sailfish_quant/SRR493371_quant/aux/eq_classes.txt
Entered function


In [52]:
len(dic_uniq_transcripts)

106186

In [53]:
len(dict_eqClass)

118317

In [54]:
lst_uniq_tids = dic_uniq_transcripts.keys()
dict_prob = {}
lst_keys = dict_eqClass.keys()
for tup in lst_keys:
    for t1,t2 in itertools.combinations(tup,2):
        set_t1 = dic_uniq_transcripts[t1]
        set_t2 = dic_uniq_transcripts[t2]
        length_intersection = len(set_t1.intersection(set_t2))
        length_union = len(set_t1.union(set_t2))
        if length_intersection > 0:
            dict_prob[(t1,t2)] = length_intersection / length_union

In [46]:
# from operator import itemgetter
# dict_prob

In [77]:
condition2_data = dict_prob
condition1_data

{(64825, 64829): 0.25,
 (51631, 51638): 0.3333333333333333,
 (53454, 53458): 0.2,
 (56340, 56345): 0.2857142857142857,
 (54334, 54336): 0.19047619047619047,
 (63843, 63853): 0.16666666666666666,
 (54353, 54365): 0.4,
 (67679, 67685): 0.35714285714285715,
 (72220, 72233): 0.13333333333333333,
 (62982, 62985): 0.1111111111111111,
 (54472, 54474): 0.5,
 (68040, 68041): 0.18181818181818182,
 (45362, 45385): 0.09523809523809523,
 (21039, 21040): 0.5,
 (62599, 62603): 0.24,
 (69761, 69771): 0.21052631578947367,
 (63398, 63400): 0.2,
 (76549, 76552): 0.14285714285714285,
 (65527, 65534): 0.6842105263157895,
 (78937, 78942): 0.38461538461538464,
 (61592, 61593): 0.3333333333333333,
 (76226, 76233): 0.5,
 (71501, 71513): 0.5454545454545454,
 (67096, 67099): 0.5,
 (49550, 49554): 0.3333333333333333,
 (70314, 70316): 0.2857142857142857,
 (70586, 70601): 0.5,
 (41753, 41754): 0.1,
 (78672, 78674): 0.2,
 (64896, 64898): 0.5,
 (60461, 60477): 0.6666666666666666,
 (75630, 75637): 0.5,
 (62315, 62322)

In [56]:
import csv
def writeToCSV(filename, mydict):
    print (filename)
    with open(filename, 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in mydict.items():
            writer.writerow([key, value])

In [57]:
writeToCSV('condition2_probability_success_ver1.2.csv', dict_prob)

condition2_probability_success_ver1.2.csv


In [50]:
# condition1_successProb = pd.read_csv('condition1_probability_success.csv')
# condition2_successProb = pd.read_csv('condition2_probability_success.csv')


In [58]:
probSuccess_diff  = {}
for k,cond1_val in condition1_data.items():
    if k in condition2_data:
        cond2_val = condition2_data[k]
        diff = abs(cond2_val - cond1_val)
        probSuccess_diff[k] = diff

In [59]:
probSuccess_diff

{(64825, 64829): 0.08333333333333334,
 (51631, 51638): 0.0,
 (56340, 56345): 0.0,
 (54334, 54336): 0.0165631469979296,
 (63843, 63853): 0.012820512820512803,
 (54353, 54365): 0.04285714285714287,
 (67679, 67685): 0.09740259740259738,
 (72220, 72233): 0.020512820512820523,
 (62982, 62985): 0.0,
 (67912, 67914): 0.0,
 (68040, 68041): 0.015151515151515166,
 (45362, 45385): 0.0082815734989648,
 (62599, 62603): 0.0,
 (69761, 69771): 0.0,
 (76549, 76552): 0.038961038961038974,
 (65527, 65534): 0.052631578947368474,
 (78937, 78942): 0.03205128205128205,
 (76226, 76233): 0.0,
 (71501, 71513): 0.054545454545454564,
 (67096, 67099): 0.125,
 (49550, 49554): 0.06666666666666671,
 (70314, 70316): 0.0,
 (70586, 70601): 0.020000000000000018,
 (41753, 41754): 0.024999999999999994,
 (65983, 65984): 0.14285714285714285,
 (60461, 60477): 0.0,
 (75630, 75637): 0.0,
 (62315, 62322): 0.0,
 (62154, 62166): 0.07500000000000001,
 (21923, 21924): 0.16666666666666669,
 (71763, 71765): 0.0,
 (55417, 55434): 0.032

In [78]:
filterGraph = pd.read_csv('filterGraph.txt', sep='\t')

In [60]:
len(tnames)

107389

In [80]:
new_probSuccess = {}
with open('new_output_ver1_3.txt', 'w') as ofile:
    for k,v in probSuccess_diff.items():
        if v < 0.2 :   
            t1_name = tnames[k[0]]
            t2_name = tnames[k[1]]
            new_probSuccess[(t1_name, t2_name)] = v
            ofile.write("{}\t{}\t{}\n".format(t1_name, t2_name, v))

In [81]:
len(new_probSuccess)

135585

In [82]:
len(probSuccess_diff)

137005